In [36]:
import pandas as pd
import numpy as np
import os
import json
import re
import time
import requests

In [46]:
file_path = './States/California/Sacramento/Sacramento_listings_results_grid_1_to_120.csv'
df = pd.read_csv(file_path)
df.columns

Index(['Room_id', 'Listing_url', 'Next_30_days_booked_days',
       'Next_30_to_60_days_booked_days', '75_rule_met', '55_rule_met',
       'Available_dates_by_year_and_month', 'Review_count_by_year_and_month',
       'Warning_level', 'Warning_type', 'Warning_message', 'Rating',
       'Accuracy_rating', 'Checking_rating', 'Cleanliness_rating',
       'Communication_rating', 'Location_rating', 'Value_rating',
       'Review_count', 'Review_months_this_year', 'Review_months_last_year',
       'Missing_review_months_this_year', 'Missing_review_months_last_year',
       'Total_missing_review_months_this_year',
       'Total_missing_review_months_last_year', 'Is_superhost', 'Guest_count',
       'Bedroom_count', 'Bed_count', 'Bath_count', 'Amenities', 'Co_hosts',
       'Highlights', 'Is_guest_favorite', 'Title', 'Latitude', 'Longitude',
       'Grid_index'],
      dtype='object')

In [47]:
print(f"Before sanitization: {df['Bath_count'].unique()}\n")

df.loc[df['Bath_count'] == 'Dedicated', 'Bath_count'] = 1
df.loc[df['Bath_count'] == 'Private', 'Bath_count'] = 1
df.loc[df['Bath_count'] == 'Shared', 'Bath_count'] = 1
df.loc[df['Bath_count'] == 'No', 'Bath_count'] = 0
df['Bath_count'] = df['Bath_count'].astype(float)

print(f"After sanitization: {sorted(df['Bath_count'].unique())}")

Before sanitization: ['1' '4' '2' '1.5' '2.5' '3' '3.5' 'Dedicated' 'Private' 'Shared' '4.5'
 'No' '0' '8' '5' '17']

After sanitization: [0.0, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 8.0, 17.0]


In [48]:
df.to_csv(file_path, index=False)
print(f"Sanitized file saved to {file_path}")

Sanitized file saved to ./States/California/Sacramento/Sacramento_listings_results_grid_1_to_120.csv


# Test

In [92]:
test_file_path = './States/California/Napa/Napa_listings_results_grid_1_to_132.csv'
test_df = pd.read_csv(test_file_path)

import ast
import datetime

def expand_review_counts(df, n_years=4):
    col='Review_count_by_year_and_month'

    current_year = datetime.datetime.now().year
    all_years = set()
    expanded = []
    for val in df[col].fillna('{}'):
        try:
            parsed = ast.literal_eval(val)
        except Exception:
            parsed = {}
        for year_key in parsed.keys():
            # Try to keep only numeric string keys
            try:
                int(year_key)
                all_years.add(year_key)
            except Exception:
                continue
        expanded.append(parsed)
    if all_years:
        max_year = max([int(y) for y in all_years])
    else:
        max_year = current_year
    # Only keep the four most recent years from max_year
    selected_years = [str(y) for y in range(max_year, max_year-n_years, -1)]
    month_names = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    # Prepare column names, only for four most recent years
    colnames = []
    for y in selected_years:
        for i, m in enumerate(month_names):
            colnames.append(f'reviews_{y}_{m}')
    # Initialize data
    data = []
    for parsed in expanded:
        row = []
        for y in selected_years:
            vals = parsed.get(y, [0]*12)
            for i in range(12):
                if isinstance(vals, list) and i < len(vals):
                    row.append(vals[i])
                else:
                    row.append(0)
        data.append(row)
    new_df = pd.DataFrame(data, columns=colnames, index=df.index)
    df = pd.concat([df, new_df], axis=1)
    return df

# Apply function to expand review counts into columns, keeping only the four most recent years
test_df_expanded = expand_review_counts(test_df, n_years=5)

test_df_expanded.shape

(402, 98)

In [93]:
import pandas as pd

def filter_by_min_reviews(
    df,
    start_year: int,
    start_month: str,
    end_year: int,
    end_month: str,
    min_reviews_per_month: int
):
    """
    Filter rows by requiring at least `min_reviews_per_month` in every month column
    between (and including) start_year/start_month and end_year/end_month.

    - start_month and end_month are three-letter capitalized: 'Jan',..., 'Dec'
    - Year is int, eg. 2023.
    """

    # Map months to numbers
    month_names = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    month_to_num = {m: i for i, m in enumerate(month_names)}

    # Ensure month names are valid
    if start_month not in month_to_num or end_month not in month_to_num:
        raise ValueError("Month must be one of: " + ", ".join(month_names))
    
    # Assemble full list of (year, month) pairs in the inclusive range
    tmplist = []
    for y in range(start_year, end_year+1):
        start_m = 0 if y > start_year else month_to_num[start_month]
        end_m = 11 if y < end_year else month_to_num[end_month]
        for m_idx in range(start_m, end_m+1):
            tmplist.append((y, month_names[m_idx]))

    # Determine column names to use
    colnames = [f"reviews_{y}_{m}" for (y, m) in tmplist]

    # Only keep columns that are in the DataFrame
    colnames = [c for c in colnames if c in df.columns]

    # For each row, require all monthly review counts in the period to be >= min_reviews_per_month
    filtered_df = df[(df[colnames] >= min_reviews_per_month).all(axis=1)]

    return filtered_df

# Example usage:
results = filter_by_min_reviews(
    test_df_expanded, 
    start_year=2023, 
    start_month="Jan", 
    end_year=2025, 
    end_month="Dec", 
    min_reviews_per_month=2
)
results

,Room_id,Listing_url,Next_30_days_booked_days,Next_30_to_60_days_booked_days,75_rule_met,55_rule_met,Available_dates_by_year_and_month,Review_count_by_year_and_month,Warning_level,Warning_type,...,reviews_2022_Mar,reviews_2022_Apr,reviews_2022_May,reviews_2022_Jun,reviews_2022_Jul,reviews_2022_Aug,reviews_2022_Sep,reviews_2022_Oct,reviews_2022_Nov,reviews_2022_Dec
60,47797528,https://www.airbnb.com.sg/rooms/47797528,13,1,False,False,"{'2026': {'1': [10, 11, 12, 13, 14, 15, 16, 17...","{'2025': [7, 6, 5, 9, 9, 10, 9, 8, 8, 8, 9, 6]...",NaN,NaN,...,10,9,14,11,10,6,9,13,8,9
151,35517924,https://www.airbnb.com.sg/rooms/35517924,3,2,False,False,"{'2026': {'1': [5, 6, 7, 8, 9, 10, 11, 12, 13,...","{'2025': [4, 7, 7, 9, 5, 8, 9, 8, 5, 7, 7, 3],...",NaN,NaN,...,11,10,7,13,13,8,7,11,8,7
152,39899839,https://www.airbnb.com.sg/rooms/39899839,2,2,False,False,"{'2026': {'1': [5, 6, 7, 8, 9, 10, 11, 12, 13,...","{'2025': [4, 3, 6, 8, 11, 9, 9, 12, 9, 14, 6, ...",NaN,NaN,...,10,11,14,11,12,14,19,13,9,10
193,49469517,https://www.airbnb.com.sg/rooms/49469517,5,7,False,False,"{'2026': {'1': [5, 6, 7, 8, 10, 11, 12, 13, 14...","{'2025': [2, 7, 6, 9, 5, 13, 13, 9, 11, 7, 11,...",NaN,NaN,...,12,10,12,13,11,13,9,11,9,10
196,48963933,https://www.airbnb.com.sg/rooms/48963933,8,10,False,False,"{'2026': {'1': [3, 4, 5, 6, 7, 8, 9, 10, 11, 1...","{'2025': [5, 5, 7, 7, 3, 8, 7, 7, 7, 5, 5, 5],...",NaN,NaN,...,7,7,8,9,11,9,9,7,7,5
199,2302352,https://www.airbnb.com.sg/rooms/2302352,9,6,False,False,"{'2026': {'1': [4, 5, 6, 7, 8, 9, 11, 12, 13, ...","{'2025': [8, 5, 10, 5, 8, 8, 6, 7, 6, 8, 6, 3]...",NaN,NaN,...,8,5,12,11,11,10,8,4,8,8
229,6914167,https://www.airbnb.com.sg/rooms/6914167,16,13,False,False,"{'2026': {'1': [4, 5, 6, 7, 8, 9, 10, 11, 12, ...","{'2025': [6, 4, 6, 5, 5, 4, 5, 5, 7, 5, 5, 3],...",NaN,NaN,...,6,8,9,9,9,8,12,5,4,4
259,49462959,https://www.airbnb.com.sg/rooms/49462959,1,5,False,False,"{'2026': {'1': [3, 4, 5, 6, 7, 8, 9, 10, 11, 1...","{'2025': [2, 2, 5, 6, 8, 9, 12, 13, 10, 6, 6, ...",NaN,NaN,...,8,9,13,11,10,11,12,10,12,5
315,9430208,https://www.airbnb.com.sg/rooms/9430208,9,6,False,False,"{'2026': {'1': [4, 5, 6, 7, 8, 9, 11, 12, 13, ...","{'2025': [6, 4, 5, 8, 8, 5, 8, 9, 8, 7, 5, 3],...",NaN,NaN,...,7,7,9,12,10,9,12,7,4,3
316,13411242,https://www.airbnb.com.sg/rooms/13411242,6,0,False,False,"{'2026': {'1': [7, 8, 9, 11, 12, 13, 14, 15, 1...","{'2025': [4, 4, 3, 6, 6, 4, 2, 7, 6, 5, 5, 2],...",NaN,NaN,...,12,6,9,14,8,10,12,8,9,7


In [94]:
for col in test_df_expanded.columns:
    if "reviews_" in col:
        print(col)

reviews_2026_Jan
reviews_2026_Feb
reviews_2026_Mar
reviews_2026_Apr
reviews_2026_May
reviews_2026_Jun
reviews_2026_Jul
reviews_2026_Aug
reviews_2026_Sep
reviews_2026_Oct
reviews_2026_Nov
reviews_2026_Dec
reviews_2025_Jan
reviews_2025_Feb
reviews_2025_Mar
reviews_2025_Apr
reviews_2025_May
reviews_2025_Jun
reviews_2025_Jul
reviews_2025_Aug
reviews_2025_Sep
reviews_2025_Oct
reviews_2025_Nov
reviews_2025_Dec
reviews_2024_Jan
reviews_2024_Feb
reviews_2024_Mar
reviews_2024_Apr
reviews_2024_May
reviews_2024_Jun
reviews_2024_Jul
reviews_2024_Aug
reviews_2024_Sep
reviews_2024_Oct
reviews_2024_Nov
reviews_2024_Dec
reviews_2023_Jan
reviews_2023_Feb
reviews_2023_Mar
reviews_2023_Apr
reviews_2023_May
reviews_2023_Jun
reviews_2023_Jul
reviews_2023_Aug
reviews_2023_Sep
reviews_2023_Oct
reviews_2023_Nov
reviews_2023_Dec
reviews_2022_Jan
reviews_2022_Feb
reviews_2022_Mar
reviews_2022_Apr
reviews_2022_May
reviews_2022_Jun
reviews_2022_Jul
reviews_2022_Aug
reviews_2022_Sep
reviews_2022_Oct
reviews_2022_N